Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

In [2]:
?VB

search: 

A Variational Bayesian modeling approach. Models are defined by  bipartite graphs in which factors defining terms in the variational objective are connected to nodes defining variables.


Let's try to make a Gaussian node with several different sizes and parameterizations:

In [3]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

VB VBModel A_rdiv_Bt A_rdiv_Bc A_ldiv_Bt A_ldiv_Bc A_ldiv_B! At_rdiv_B



3

In [4]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [5]:
μ[1]

3-element Array{Float64,1}:
 0.693778
 0.846083
 0.285281

In [6]:
Σ[1]

3x3 Array{Float64,2}:
 1.11992   0.430546  0.897836
 0.430546  0.932812  0.864311
 0.897836  0.864311  1.07162 

In [7]:
det(Σ[2])

0.0006307935080160916

In [8]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...)

5x5 Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}:
 FullNormal(
dim: 3
μ: [0.6937780686967863,0.8460827861341069,0.2852809669274383]
Σ: 3x3 Array{Float64,2}:
 1.11992   0.430546  0.897836
 0.430546  0.932812  0.864311
 0.897836  0.864311  1.07162 
)
    …  FullNormal(
dim: 3
μ: [0.37379098119520604,0.09685385334345331,0.3521010149172945]
Σ: 3x3 Array{Float64,2}:
 0.876821  1.00603  1.07148
 1.00603   1.20758  1.28356
 1.07148   1.28356  1.36461
)
        
 FullNormal(
dim: 3
μ: [0.3087986313106983,0.5336099483043908,0.7880573717722976]
Σ: 3x3 Array{Float64,2}:
 0.658865  0.678777  0.484801
 0.678777  0.737977  0.350301
 0.484801  0.350301  0.956508
)
       FullNormal(
dim: 3
μ: [0.5712514622041094,0.9946211213990779,0.30063262516487543]
Σ: 3x3 Array{Float64,2}:
 0.240004  0.250571  0.28831 
 0.250571  1.00154   0.866256
 0.28831   0.866256  1.43344 
)
   
 FullNormal(
dim: 3
μ: [0.9905728302141985,0.2856840126620319,0.320741175856051]
Σ: 3x3 Array{Float64,2}:
 1.68521

In [9]:
rr = RandomNode(aa, Factor[]);

In [10]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [11]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [12]:
ss = RandomNode(MultivariateNormal, pars...);

In [13]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.6937780686967863,0.8460827861341069,0.2852809669274383]    …  [0.37379098119520604,0.09685385334345331,0.3521010149172945]
 [0.3087986313106983,0.5336099483043908,0.7880573717722976]       [0.5712514622041094,0.9946211213990779,0.30063262516487543] 
 [0.9905728302141985,0.2856840126620319,0.320741175856051]        [0.5134554738571968,0.6458511750969451,0.5701092446222313]  
 [0.22606701150402198,0.8210840359088869,0.22819730313042408]     [0.49357423824578484,0.9940624338815678,0.4151485652228162] 
 [0.7289226272079101,0.4882246131718575,0.29955869567838844]      [0.3174759484605427,0.8514061901130703,0.7502074052500889]  

In [14]:
?E

search: 

Calculates the expected value of a Node x.


In [15]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}

# Make a factor and check its value

In [16]:
μ = rand(10)
τ = rand(10)
x = RandomNode(Normal, rand(10), rand(10))
f = LogNormalFactor(x, μ, τ)

VB.LogNormalFactor(VB.RandomNode{Distributions.Normal,1}([Distributions.Normal(μ=0.21053116128424132, σ=0.3566024518831339),Distributions.Normal(μ=0.891285361977286, σ=0.6442949243350777),Distributions.Normal(μ=0.9190711480513318, σ=0.5847535950545695),Distributions.Normal(μ=0.8357027050511385, σ=0.9588584490678307),Distributions.Normal(μ=0.7011301523093227, σ=0.11988256254636265),Distributions.Normal(μ=0.4936985944104011, σ=0.49515232739001),Distributions.Normal(μ=0.43476857161658544, σ=0.06006314188566453),Distributions.Normal(μ=0.9057559221849028, σ=0.5939495004821511),Distributions.Normal(μ=0.08371485686139835, σ=0.6412695407108462),Distributions.Normal(μ=0.09628303073445155, σ=0.8367931299182829)],VB.Factor[]),VB.ConstantNode{Float64,1}([0.8960777324549798,0.2703005448728435,0.6288387723162663,0.5732161023467082,0.8203815274260202,0.9540175015140073,0.8874567679777285,0.02546966967547748,0.04665785891317231,0.7887935071010339],VB.Factor[]),VB.ConstantNode{Float64,1}([0.49798076214

E e eu ENV end Expr eye exp eta esc erf eps eof eig Enum Elog else expm



In [17]:
value(f)

10-element Array{Float64,1}:
 -0.719024
 -0.966352
 -0.641224
 -0.927615
 -0.104573
 -1.09986 
 -0.837185
 -0.428248
 -0.141838
 -0.783547

In [18]:
VB.var(f.x)

10-element Array{Float64,1}:
 0.127165  
 0.415116  
 0.341937  
 0.91941   
 0.0143718 
 0.245176  
 0.00360758
 0.352776  
 0.411227  
 0.700223  